### LX 496/796  Formalizing meaning, part II: SHRDLU


A quick word about SHRDLU.  [SHRDLU](https://en.wikipedia.org/wiki/SHRDLU) is Terry Winograd's program from 1971 that could converse about things in its very limited universe of a block world.  You can find a video of it doing its thing [on YouTube here](https://www.youtube.com/watch?v=bo4RvYJYOzI).  You can see it doing a lot of pretty interesting things.  Learning names for things, remembering what it was talking about, moving blocks around, making plans to accomplish goals.  Ours will not be that sophisticated.

What we're going to do here is make a "little" version of this.  It's still 
actually pretty extensive, but there will still be a lot left to do by the
time we're done.

There are few different separable parts of this endeavor, some more related
to Python than they are to NLTK, but by the end you should have a cool little
program you can interact with in a limited way.

The basic parts of the program are:

- representation of the objects in the world
- grammar for syntactic parsing and semantic composition
- display module to show the current state of the world
- user input loop
- interpretation of user input to respond

The goal by the end is going to be for "SHRDLU" to be able to
answer questions like "Is the red block on an even square" and
perform actions like "put a pyramid on a block". 

*Note:* There is a lot of reading here, and in most cases I'm giving you
the pieces of the program.  It should generally be possible to copy and
paste from this web page into the Python text editor so it does not all
need to be typed.  But even if you are copying and pasting it you should
be trying to understand what it is doing, why it is written that way,
how it works.

This continues from the prior homework.

## Setting up the world ##


We'll start off by setting up the world.  This is the same as what we had in the previous homework.  Remind yourself what's happening here.

In [ ]:
import nltk
from nltk import grammar
squares = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8']
dom = {'a', 'b', 'c', 'd', 'e'} | set(squares)
valstr = """
square => {s1, s2, s3, s4, s5, s6, s7, s8}
odd => {s1, s3, s5, s7}
even => {s2, s4, s6, s8}
block => {a, b}
pyramid => {c, e}
table => {d}
thing => {a, b, c, d, e}
red => {a}
blue => {b, e}
green => {c, d}
on => {(a,s1),(b,s2),(d,s4),(c,d)}
held => {e}
"""
val = nltk.sem.Valuation.fromstring(valstr)
m = nltk.Model(dom, val)
g = nltk.Assignment(dom)

This completely specifies the world now.  We have 13 objects, 8 of which are
squares that represent the floor (4 of which are odd, 4 of which are even),
and 5 of which are shapes of various kinds (block, pyramid, table) with various
properies (red, big, etc.).  Three of the objects are on the floor, one object
is on another one, and one is in the robot hand.

## Setting up the grammar

Next up, we're going to set up a basic grammar and semantics, along the lines of what we had in the previous part of the homework, and then we will extend it.

The plan: We want to be able to say to "a green pyramid" or "every odd square" and have it figure out what object(s) are being referred to.  We want to be able to say "a table is on an even square" (a declarative sentence) and have it evaluate whether it is true or not, and we want to be able to ask "is a table on an even square" (a yes-no question) and have it detect that it is a question and answer yes or no.  And we want to be able to say "put the green pyramid on an odd square" (an imperative) and have it adjust the world accordingly.

It turns out that the noun phrases are going to be the most complicated part of this (in fact, the determiners like "the" and "every" most of all), so let's try to build up the rest of this first.  In order to escape the need to define NPs and the semantics within them, let's name a couple of the objects so we can refer to them by name.

### The architecture

In order to parse a sentence, we need
 - a model (domain and valuation function) and an assignment
 - a grammar (which specifies the semantics for words and non-terminal nodes)
 - a parser (which resolves the words into a structure)

In order to interact with the robot, we need
 - a place to type
 - evaluation of the sentence typed
 - specification of actions to take depending on the meaning of the sentence
 
In order to allow us to continue to extend this as we go along, we are
going to define a *dictionary* of grammar fragments called `gramspec`.
We will name each fragment so that we can easily update that fragment.
And then for parsing a sentence, we will define a function that
assembles the fragments into a single string, creates a grammar from it,
and then creates a parser for that grammar.

Let's set this up a bit.  The first two things we will add to our grammar
are:
 - the definition of the top of the tree, which is CP, and
 - names for two specific objects in our world.


First, we will call the green pyramid (c) "pat" and the green table (d) "chris".

In [ ]:
# create the empty grammar specification dictionary
gramspec = {}
# Add the specification for the name definition
gramspec['name'] = r"""
DP[SEM=<pat>] -> 'pat'
DP[SEM=<chris>] -> 'chris'
"""

And then we will define CP.  Remember that the `% start CP` line has to be up at the top of the overall grammar specification string, it defines what the symbol is at the top of the tree.

> Also, it is probably worth taking a second to talk about the CP rules below themselves.  Remember how these rules work from the previous homework.  The parts in square brackets define features, and we are defining two features here.  One is `SEM` (for "SEMantics") and one is `CT` (for "Clause Type").  The `?cbar` and `?ct` and `?tp` are variables.  They name the values that appear on the right, and they are used to set values on the left.  So the first rule says "Let's call the `CT` value (from `CBAR`) `?ct` and the `SEM` value (from `CBAR`) `?cbar`.  We can build a `CP` out of a `CBAR` (this corresponds to a tree that has just a single branch from CP down to CBAR, no specifier), and the `SEM` value of `CP` will be `?cbar` (so, just the same as that value was for `CBAR`) and the `CT` value of `CP` will be `?ct` (so, just the same as that value was for `CBAR`).  Essentially, the `SEM` and `CT` features are being "passed up" from `CBAR` to `CP`.  That's what the first rule says.  The second rule says that the `SEM` and `CT` features are being passde up from `TP` to `CBAR` too.  So, this is a lot of nothing, semantically speaking, but we may be tweaking some of this later.

In [ ]:
# define CP as containing Cbar and TP, with trivial semantics
gramspec['cp'] = r"""
% start CP
CP[SEM=?cbar, CT=?ct] -> CBAR[SEM=?cbar, CT=?ct]
CBAR[SEM=?tp, CT=?ct] -> TP[SEM=?tp, CT=?ct]
"""

We can't build a *useful* grammar out of just these two fragments yet, but we can still see how we will be filling in more fragments until we have a workable grammar.  Note too that defining them like `gramspec['cp'] = ...` means that later, if/when we want to change just the CP part of the grammar, we can just update `gramspec['cp']` and it will be integrated into the grammar.  It seems cleaner, division and conquest.

We can define the following function to assemble the grammar specification string:

In [ ]:
def assemble_spec(gramspec):
    return "".join(sorted(gramspec.values()))

This just concatenates all the fragments.  Two notes:
 - `.values()` gives us just the values and not the keys from the dictionary.  So just the rules, not the label like 'cp'.
 - `sorted()` is a sneaky trick because we need to get the `% start CP` up at the top, and the fragment that starts with `%` will be alphabetically sorted before anything else we have.
 
When we try it, we get:

In [ ]:
print(assemble_spec(gramspec))

To parse a sentence, we need to construct a grammar, a parser, break the sentence into words, and parse it.  We can put that into a function that does all of that at once, and returns the list of parses.

In [ ]:
# parse a sentence to compute its semantic value(s)
def sent_parse(gramspec, sentence):
    full_spec = assemble_spec(gramspec)
    new_grammar = grammar.FeatureGrammar.fromstring(full_spec)
    parser = nltk.FeatureChartParser(new_grammar)
    words = sentence.split()
    return list(parser.parse(words))

We don't yet have a complete enough grammar to parse anything, so we can't test this out quite yet, but it's a little easier to put it in here up at the top before we get into the details of the grammar itself.

Another thing we will want to do is evaluate whether the semantics we get are true of our model of the world, so we can define a function that does that now as well.  Because there might be several parses in a list (notice that `sent_parse` returns a list of parses), this will evaluate all of the semantic expressions in a list.

In [ ]:
# evaluate whether a list of expressions are true in the model
def eval_sent(m, g, expressions):
    return [m.satisfy(expr, g) for expr in expressions]

This one we can test if we make our own expression by hand.

In [ ]:
# test eval_sent out on 'there is a green pyramid' and 'there is a red pyramid'
expr1 = nltk.Expression.fromstring('exists x.(green(x) & pyramid(x))')
expr2 = nltk.Expression.fromstring('exists x.(red(x) & pyramid(x))')
eval_sent(m, g, [expr1, expr2])

And, lastly for the moment, to mediate between `sent_parse` and `eval_sent` we can define `sent_sem` that will construct a list of parses and extract the `SEM` feature from the top of them (the semantic representation of the entire sentence) to make expressions (which we can then give to `eval_sent` to evaluate against the model).  This doesn't really do very much, it just will make what we do later marginally more readable.

In [ ]:
# parse a sentence and return only the overall semantics (of top node)
def sent_sem(gramspec, sent):
    parses = sent_parse(gramspec, sent)
    return [x.label()['SEM'] for x in parses]

Again, we can't test this yet because we can't parse anything yet, but it is clear that it collects the parses (using `sent_parse`) and then gets the `SEM` feature from the top of the parse (the `label()` of the parse).

One other useful thing is that if you want to see what fragments have been added to the specification dictionary, you can submit it to `list()`.  This will give you a list of just the keys.  (And if you want just the values without the keys, you can use the `values()` method, like we saw in the definition of `assemble_spec` above.)

In [ ]:
# display the keys in the dictionary (the fragment names)
list(gramspec)

### Getting a simple grammar running

Above, we added fragments for CP and two names to the grammar, with the intention that they will refer to specific objects in our model, but we have not yet added them to our model.  So we need to say what "pat" and "chris" refer to.  This information about the model/world is in `val` so we need to update `val` so it knows what individuals "pat" and "chris" are. 

In [ ]:
val['pat'] = 'c'
val['chris'] = 'd'

Just to make sure everything looks as we expect, we can print out the valuation function as it stands now.  This should look familiar, but now with more pat and chris than before.

In [ ]:
print(val)

### Defining the VP and PP (the predicates)

In this grammar, we're actually only going to have one kind of sentence, so we can take a shortcut.  The only verbs we'll have (at least before we get to the imperative verbs) we be "be" plus a PP (like "on the table").  And if you think about what "be" contributes in that case, it is... well, it is nothing.  The real predicate in "a block is on the table" is "on".  So, we'll define a VP that has "is" as its verb, and adds no meaning at all, just passes up the meaning of the PP contained within it.

In [ ]:
# define VP as being "is" plus a PP, with the same semantics as the PP
gramspec['vp'] = r"""
VP[SEM=?pp] -> VCOP PP[SEM=?pp]
VCOP -> 'is'
"""

And then define PP as being combined of the semantics of the P (like "on") and the semantics of the DP object.  As a reminder, the way you read this is that the `?p` on the right side and the `?dp` on the right side are variable labels, and then we use those labels on the left side.  So this is saying: Call the semantics of the P `?p` and call the semantics of the DP `?dp`.  The semantics of the combination (PP) is going to be what you get when you apply `?p` to `?dp`.

In [ ]:
# define PP as applying the function P to the argument DP
gramspec['pp'] = r"""
PP[SEM=<?p(?dp)>] -> P[SEM=?p] DP[SEM=?dp]
"""

We then need to define what the Ps are.  We have one P relation defined in the model of the world, "on".  "On" is a two-place predicate, it is a relation between two individual objects.  To determine whether "on" gives us true or false, we need to get two individuals (call them `x` and `y`), and then check to see if `(x,y)` is in the definition of `on` in the model.

Breaking this down, we want "on pat" to be true of anything `y` such that `(y,c)` is in the list of pairs that define `on`.  So the `SEM` value of that PP should be:

```
\y.on(y,c)
```

...which is true of any things that are on `c` (pat).  Now, looking inside the PP, it is made of "on" and "pat", and we presumably want to get the `c` part there from "pat", so we want to factor that out of the definition of the P "on".  That is, we want the P "on" to be something that, if given an `x` (like "pat"), it will yield the function above.

```
\x.\y.on(y,x)
```

In [ ]:
# define the two-place predicate 'on'
gramspec['p'] = r"""
P[SEM=<\x.\y.on(y,x)>] -> 'on'
"""

And then, to complete a (comically simple) grammar, we can define the part at the top of the tree that combines the subject and the verb.
We already defined CP up at the beginning, but we also need to define TP.

The definition of TP is (for now) going to take the function provided by the VP and apply it to the subject DP.  This is basically familiar from last time.

In [ ]:
# define TP as taking the VP predicate as a function applied to the DP subject as its argument
gramspec['tp'] = r"""
TP[SEM=<?vp(?subj)>, CT='decl'] -> DP[SEM=?subj] VP[SEM=?vp]
"""

We now have a grammar just barely sufficient to parse a couple of sentences.  We can parse "pat is on chris" and "chris is on pat" if everything worked ok.  But, sure, it's a start.  Let's try it out.  First, we'll have a look at our grammar specification so far, and then get the parses.

In [ ]:
# look at our grammar, assembled from fragments
print(assemble_spec(gramspec))

In [ ]:
# get a list of semantic values for parses for 'pat is on chris', print them, and evaluate them
parse_sems = sent_sem(gramspec, 'pat is on chris')
print(parse_sems)
print(eval_sent(m, g, parse_sems))

**TASK 1**. Demonstrate that it is not true that chris is on pat.

In [ ]:
# Answer 1.  Show that 'chris is on pat' is false in the model


**TASK 2**. Use `sent_parse` (defined a little while back, and used within `sent_sem`) to get the clause type feature (`CT`) of the sentence "pat is on chris".   (It will of course be `'decl'` but the point is to show how to retrieve that information.)

> We are always working with lists of parses (e.g., `sent_sem()` above is returning a list of the semantic values, one per parse), even though those lists are generally just one parse long for everything we are going to be doing in this homework.  So, you can assume that the first parse you get back (if you get any back) is the only relevant one.

In [ ]:
# Answer 2.  Retrieve the CT feature of the parsed sentence "pat is on chris"


Ok, not bad.  We now have a grammar and semantics that can handle our two named individuals and the "on" relation.  It can parse sentences.  We are on our way.

## Handling quantifiers, part one

The next thing we are going to do is try to handle quantifiers like "a pyramid" or "every square".
But this is actually kind of complicated, and it requires kind of turning how we think about this upside down.
Let's see if we can make sense of this with Python functions, and then translate it to semantics.
So we are going to leave the blocks world for a bit and just work with an even simpler setup,
not using NLTK, just using Python functions and arguments to model semantic combinations.

In [ ]:
# these are the individuals
pat = 'p'
chris = 'c'
tracy = 't'
fido = 'f'
# all_individuals is the entire set of individuals
all_individuals = {pat, chris, tracy, fido}
# these are the properties
dogs = {fido}
people = {pat, chris, tracy}
swimmers = {pat, chris}
singers = {pat, chris, tracy}
# the verb swim is true of individuals that are swimmers.
def swim(x):
    return x in swimmers
# the verb sing is true of individuals that are readers.
def sing(x):
    return x in singers
# so pat swims is true and tracy swims is false and tracy sings is true.
pat_swims = swim(pat)
tracy_swims = swim(tracy)
tracy_sings = sing(tracy)
# a person swims
a_person_swims = True in [swim(x) for x in people]
# every person swims
every_person_swims = not False in [swim(x) for x in people]
# every person sings
every_person_sings = not False in [sing(x) for x in people]
print("pat swims: {}".format(pat_swims))
print("tracy swims: {}".format(tracy_swims))
print("tracy sings: {}".format(tracy_sings))
print("a person swims: {}".format(a_person_swims))
print("every person swims: {}".format(every_person_swims))
print("every person sings: {}".format(every_person_sings))

So, above we have defined `swims` as a function that will return `True` or `False` depending on whether the person we provide (`x`) is in the set of `swimmers`.
That feels right for the definition of the verb "swims".

But if we want to determine whether a person swims we need to go through all the people, check for each whether they swim, and then see if we got `True` for at least one of them.
And for "every", same thing except we need to see if we got `True` for all of them (or, equivalently, never got `False`).

The next challenge is to factor out "person" so we can define a semantics for "a" and "every" alone.  There's a pretty natural way to define "person".  And "dog" for that matter.

**TASK 3**.  Define `person(x)` and `dog(x)` in that natural, obvious way, basically on the model of `swim(x)` above.

In [ ]:
# Answer 3.  Define persion(x) and dog(x)


Assuming that worked, we should wind up with pat being a person, and fido being not a person but a dog.

In [ ]:
# fido is not a person, pat is a person. fido is a dog.
print("pat is a person: ", end='')
print(person(pat))
print("fido is a person: ", end='')
print(person(fido))
print("fido is a dog: ", end='')
print(dog(fido))

It is possible for a function to have a function as its argument.  As an example, a function that tells us stuff about tracy depending on what function we give it.

In [ ]:
# inquire about tracy's properties
def tracy_properties(predicate):
    return predicate(tracy)
# if we ask tracy's properties about "swim" we get false, about "person" we get true.
print(tracy_properties(swim))
print(tracy_properties(person))
# this does the same thing as printing swim(tracy) and person(tracy)

That's kind of wild.  **Understand what just happened**.  We passed in a *function*, either `swim` or `person`,
and `tracy_properties` applied whatever function we passed it to `tracy` to get the answer.

Now, to define "a" for "a person swims", what we are aiming for is something like:
you can find an `x` among our individuals (`all_individuals`) where
both `person(x)` is true and `swims(x)` is true.
So the way to figure that out is to make a list, for each individual, whether that individual
is both among the people and among the swimmers.
Afterwards, you can look at the list and if there's a `True` in there somewhere, then
such an individual exists, and "a person swims" is true.

So "a" takes two predicates (person being one, and swim being the other) to give us
"a person swims".  Generalizing, and applying that to code:

In [ ]:
# define "a" that will get us "a person swims"
def det_a(P, Q):
    return True in [Q(x) & P(x) for x in all_individuals]
# define "every" that will get us "every person swims"
def det_every(P, Q):
    return not False in [Q(x) & P(x) for x in all_individuals]
# so does a person swim?
print("A person swims: ", end='')
print(det_a(person, swim))
print("Every person swims: ", end='')
print(det_every(person, swim))

**TASK 4.** The definition of `det_every` above kind of works, but not quite.  It gives `False` for `det_every(person, sing)`.  Describe (in markdown) why this comes out as `False`.  What change (in Python) do you need to make to `det_every` to make it corrently give `True` for `det_every(person, sing)` and `False` for `det_every(person, swim)`?

*Answer 4*. (markdown)

In [ ]:
# Answer 4: Redefine det_every(P, Q) so that it works properly


Now, think about this in terms of a syntactic tree, with a subject DP ("every person") and a VP ("swims").
The subject is made of "every" and "person".
"Every" is a function that requires two predicates. The first one is "person", inside the DP.
But there is still a need for a second one.  The subject DP is a function that needs a predicate still.
We can define the function the quantified subject represents as follows,
which *might* help make it clear what is happening.
The function `quant_dp` takes a determiner (which is one of the functions we defined above, that need two predicates)
and a predicate (the noun inside the DP), and reduces the function by one place by
substituting in the noun, and returning a function that still needs one predicate.
That means that `quant_dp(det_a, person)` winds up representing a function that takes a predicate as an argument.
The VP is such a predicate, for example, `swim`.

In [ ]:
def quant_dp(Det, P):
    return lambda Q:Det(P, Q)

a_person = quant_dp(det_a, person)
print(a_person(swim))
every_person = quant_dp(det_every, person)
print(every_person(swim))

This is pretty much exactly what we are going to do in our semantics.  We are going to make determiners take two predicates,
which means that we are going to make quantified DPs like "a person" take the VP predicate.

Before we leave this, though, let's also consider what we're going to do with "on", because this gets even hairier.

The simple version of "on" that we have so far is something like the following.
We will first define who is on whom and then define a simple semantics for "on".

In [ ]:
# define who is on whom in the world
on = {(fido, tracy)}
# define a too-simple semantics for "on"
def p_on(x, y):
    return (x,y) in on
# fido is on tracy and not on pat
print(p_on(fido, tracy))
print(p_on(fido, pat))

At this point, we can already handle the semantics for the sentence "a dog is on tracy" just fine.
We'll define the semantics of the PP like we defined the semantics of the subject DP,
taking one argument and returning a function that still needs one argument.  So we can have a one-place predicate like "(be) on tracy".
Below we assemble "on tracy" and "on pat" and "a dog" and "every dog" and "every person".
And test them out by providing the predicate that the PP represents to the function that the subject quantifier represents.

In [ ]:
# define the PP (aka VP) by providing the internal argument for the P (on)
def pred_pp(y):
    return lambda x: p_on(x, y)

# create one-place predicates out of a P and its object
# (be) on tracy, (be) on pat:
pp_on_tracy = pred_pp(tracy)
pp_on_pat = pred_pp(pat)

# fido is on tracy
print(pp_on_tracy(fido))
# fido is not on pat
print(pp_on_pat(fido))

# create quantified DPs that require a one-place predicate
subj_a_dog = quant_dp(det_a, dog)
subj_every_dog = quant_dp(det_every, dog)
subj_every_person = quant_dp(det_every, person)

# we can give those the one-place predicates from above.
# a dog is on tracy
print(subj_a_dog(pp_on_tracy))
# in fact every dog is on tracy
print(subj_a_dog(pp_on_tracy))
# but it's not the case that every person is on tracy
print(subj_every_person(pp_on_tracy))

The place it gets troublesome is when we consider what to say about "a dog is on a person".
How do we define "on" in such a way that it can have such a complex function as its argument?
The answer we'll pursue is that we'll make the semantics of "on" **EVEN MORE COMPLEX** than a quantified DP.
We are going to have "on" take a quantified DP as its argument and do the appropriate thing with it.

In [ ]:
# define this complicated "on" that can take quantified objects.
def super_pred_pp(dp):
    return lambda x: dp(lambda y: p_on(x, y))
# is fido on a person?
pp_on_a_person = super_pred_pp(quant_dp(det_a, person))
print(pp_on_a_person(fido))
# is a dog on a person?
print(quant_dp(det_a, dog)(pp_on_a_person))

This is a lot to absorb. That last step was a big one.  Let me try to talk through it a little bit.

The version of "on" that we've defined in `super_pred_pp` takes a quantified DP as its argument (so, "on a person").
A quantified DP relates two predicates, the one inside to one outside.
So a quantified DP winds up representing a function that needs another predicate.
So the `dp` argument to `super_pred_pp` needs a predicate.
We provide it one, specifically the predicate "things `x` is on" for some to-be-specified `x`.
And then it returns a predicate that will be true or false of an individual.
So if we give `super_pred_pp` "a person" as its `dp` it will return a function of individuals that is
true of anything where a person can be found that it is on.  Pretty much what "on a person" means.

**TASK 5.** Using the same pieces as above, print out whether every person is on a dog.  The answer is `False`, but get that from the functions we've been working with.

In [ ]:
# Answer 5.  Using quant_dp, super_pred_pp, etc. like above, evaluate whether every person is on a dog.


## Names as quantifiers

The very last thing we'll do before heading back to the block world is to try to "simplify" this a bit
so that, instead of having two different kinds of DPs (names/individuals and quantifiers), we will
have just one kind (which will have to be quantifiers).  To do this, we essentially need to make names
be more complex, a function that takes a predicate and returns True or False.

A simple way to do this is to say that "fido" is represented by the properties true of fido.
That is, the fido function takes a property and returns true if fido has that property.
(And, actually, we already discussed `tracy_properties` above in a different context.)

In [ ]:
# convert names into quantifiers
def pat_properties(predicate):
    return predicate(pat)
def tracy_properties(predicate):
    return predicate(tracy)
def fido_properties(predicate):
    return predicate(fido)
def chris_properties(predicate):
    return predicate(chris)
# does pat swim?
print(pat_properties(swim))
# does a person swim?
print(quant_dp(det_a, person)(swim))
# is fido on tracy?
print(fido_properties(super_pred_pp(tracy_properties)))
# is a dog on a person?
print(quant_dp(det_a, dog)(super_pred_pp(quant_dp(det_a, person))))

Once the headache subsides, this is really probably easier.  We can treat DPs as having a consistent type (not functions sometimes and individuals other times).
So we can have a consistent definition of things like "on" and "swim" and so forth.

## Handling quantifiers, part two

Back to the block world and our ever-growing grammar.
How do we handle "a block" or "every pyramid"?

Following the logic we went through in the previous section, 
we want to define the determiners ("a", "every") as being
functions that take two predicates.

And we can define "pyramid" as being a predicate that is true of pyramids.

And then the DP "a pyramid" will supply the "pyramid" predicate to the determiner "a",
leaving the resulting DP with a need for one additional predicate.

In [ ]:
# define pyramid
gramspec['np'] = r"""
NP[SEM=<\y.pyramid(y)>] -> 'pyramid'
"""

# define the determiner "a"
gramspec['det'] = r"""
D[SEM=<\N Q.exists x.(N(x) & Q(x))>] -> 'a'
"""

# define DP that combines them
gramspec['dp'] = r"""
DP[SEM=<?det(?np)>] -> D[SEM=?det] NP[SEM=?np]
"""

Now that we are going to make (subject) DPs functions that take a predicate as their argument,
we redefine TP so that it applies the subject to the VP:

In [ ]:
# redefine TP for quantified subjects
gramspec['tp'] = r"""
TP[SEM=<?subj(?vp)>, CT='decl'] -> DP[SEM=?subj] VP[SEM=?vp]
"""

Although we probably will not want to keep "pat" and "chris" (our green pyramid and table) as proper names,
they are currently in our grammar, and we just removed the ability for them to appear as subjects
(because they are defined as individuals and not functions).  So, we can redefine them as functions
for the moment just so the grammar remains consistent.

In [ ]:
# redefine the names to be functions of properties
gramspec['name'] = r"""
DP[SEM=<\P.P(pat)>] -> 'pat'
DP[SEM=<\P.P(chris)>] -> 'chris'
"""

And then we need to implement the "super on" that we discussed in the previous section.
That is, we need to redefine "on" so that it takes a quantified DP as its object rather
than an individual.

In [ ]:
# redefine the P "on" to allow for quantified objects
gramspec['p'] = r"""
P[SEM=<\X x.X(\y.on(x,y))>] -> 'on'
"""

And now we can stand back and admire our work.

In [ ]:
print(assemble_spec(gramspec))

Does this do what we want?  The goal is to parse "a pyramid is on chris" into `exists x.(pyramid(x) & on(x,chris))`, and then be able to check it against the model.

In [ ]:
parse_sems = sent_sem(gramspec, 'a pyramid is on chris')
print(parse_sems)
print(eval_sent(m, g, parse_sems))

Success. Sweet.

In [ ]:
print(eval_sent(m, g, sent_sem(gramspec, "a pyramid is on pat")))

With that behind us, we can go ahead and define the grammar and semantics for all of the rest of the nouns.  They are "block", "table", "square", and "thing" (and "pyramid").
All of the rules are going to be the same form as the rule for "pyramid" was.
So, we can be a little bit tricky and save ourselves some typing (and potential typos) by building the multi-line string programmatically.
Specifically, we'll iterate over a list of the nouns, and make a rule for each one from a template.  Like so:

In [ ]:
# build the grammar fragment for the nouns
nouns = ["block", "table", "square", "pyramid", "thing"]
nounstrings = ["NP[SEM=<\\x.{n}(x)>] -> '{n}'\n".format(n=noun) for noun in nouns]
gramspec['np'] = "".join(nounstrings)
print(gramspec['np'])

## Adjectives

How about adjectives like "green" in "a green pyramid"?  So far, there is no place in the grammar for these.

Although we are again limiting ourselves to a pretty small domain, we can for now at least presume that
the adjectives we care about are "intersective".  So a green pyramid is both green and a pyramid.

> This is in contrast to something like "fake" -- it is not the case that a fake gun is both
> fake and a gun.

It is also possible to have more than one adjective
(like in "a big green pyramid").  So let's suppose that means we can in principle have
unboundedly many adjectives, and create a recursive rule that allows us to make an NP
out of an adjective and an NP.

```
NP[SEM=<...?...>] -> Adj[SEM=?adj] NP[SEM=?np]
```

So, what is the semantic value of the combination?  Well, both an NP and an Adj seem like
the same kind of property, a one-place predicate.  "Pyramid" is true of things that are
pyramids, "green" is true of things that are green.  And "green pyramid" is true of things
that are both green and pyramids.  So, it looks a little like what we had for the definition
of "a":

In [ ]:
# define NP -> Adj NP
gramspec['adjnp'] = r"""
NP[SEM=<\x.(?adj(x) & ?np(x))>] -> Adj[SEM=?adj] NP[SEM=?np]
"""

And then we can define all the adjectives, just the same way we defined the nouns.

In [ ]:
# build the grammar fragment for the adjectives
adjs = ["odd", "even", "red", "blue", "green"]
adjstrings = ["Adj[SEM=<\\x.{a}(x)>] -> '{a}'\n".format(a=adj) for adj in adjs]
gramspec['adj'] = "".join(adjstrings)
print(gramspec['adj'])

Did it work? Let's see if a green pyramid is on chris.  And if a red pyramid is on chris.  And if a green green green pyramid is on chris.  Because we can.

In [ ]:
print(eval_sent(m, g, sent_sem(gramspec, "a green pyramid is on chris")))
print(eval_sent(m, g, sent_sem(gramspec, "a red pyramid is on chris")))
print(eval_sent(m, g, sent_sem(gramspec, "a green green green pyramid is on chris")))

**TASK 6.** Show whether pat is on a red red table.

In [ ]:
# Answer 6.  Show whether pat is on a red red table.


Let's also define a couple more determiners.  The semantics of "the" and "a" and "an" are all pretty much the same
for now at least.  "The" is definite, and we might want to make use of that later.  So, as we add the determiners,
let's specify them with a definiteness feature as well.  This means that we will want to also pass that up to the DP
that combines the semantics of Det and NP.  So, we want to redefine the determiners.  But where were they again?
We can see what fragments we've defined in `gramspec` by looking at `list(gramspec)`. 

In [ ]:
print(list(gramspec))

Ok, it is `gramspec['det']` and `gramspec['dp']` that we want to redefine.  Let's remind ourselves of what was there before.

In [ ]:
print(gramspec['det'])
print(gramspec['dp'])

And then we can redefine them.  Take a look at the definition for "every" and see if you get it.  The `->` means "implies" or "entails".
If every dog swims, that means that being a dog entails swimming (hut not being a dog doesn't have any implications).

In [ ]:
# define more determiners and include the DEF feature
gramspec['det'] = r"""
D[SEM=<\N Q.exists x.(N(x) & Q(x))>, -DEF] -> 'a'
D[SEM=<\N Q.exists x.(N(x) & Q(x))>, -DEF] -> 'an'
D[SEM=<\N Q.exists x.(N(x) & Q(x))>, +DEF] -> 'the'
D[SEM=<\N Q.all x.(N(x) -> Q(x))>, -DEF] -> 'every'
"""

# redefine DP to pass DEF up to DP from Det
gramspec['dp'] = r"""
DP[SEM=<?det(?np)>, DEF=?def] -> D[SEM=?det, DEF=?def] NP[SEM=?np]
"""

Now just to verify that it worked.  We are getting to the point where we should be able to parse pretty sophisticated sentences.

In [ ]:
print(eval_sent(m, g, sent_sem(gramspec, "every pyramid is on a table")))
print(eval_sent(m, g, sent_sem(gramspec, "every green pyramid is on a table")))
print(eval_sent(m, g, sent_sem(gramspec, "every block is on a square")))
print(eval_sent(m, g, sent_sem(gramspec, "a pyramid is on the table")))
print(sent_parse(gramspec, "a green pyramid is on every table")[0].label()['SEM'])
print(sent_parse(gramspec, "a green pyramid is on every table")[0].label()['CT'])
print(eval_sent(m, g, sent_sem(gramspec, "a green pyramid is on every table")))

## Looking at the world, again

You will recall from last time that we set up a way to visualize the world.
Let's just bring that back up to the point we were at before, without further discussion.

In [ ]:
def obj_in_hand():
    if len(val['held']) == 0:
        return None
    else:
        return list(val['held'])[0][0]

def whats_on(obj):
    # ask: what is on the current support?
    f = nltk.sem.Expression.fromstring("on(x,s)")
    g2 = nltk.Assignment(dom, [('s', obj)])
    try:
        next_obj = list(m.satisfiers(f, 'x', g2))[0]
    except:
        next_obj = None
    return next_obj

def build_stack(square):
    stack = [square]
    while True:
        next_obj = whats_on(stack[-1])
        if next_obj:
            stack.append(next_obj)
        else:
            break
    return stack

# slight change here in order to allow for our named individuals chris and pat
def obj_properties(obj):
    return {v for v in val if type(val[v]) is set and (obj,) in val[v]}

def obj_shape(obj):
    shape_properties = {'block', 'pyramid', 'table', 'square'}
    shape_property = obj_properties(obj) & shape_properties
    if len(shape_property) == 0:
        return None
    else:
        return list(shape_property)[0]

def draw_simple_shape(obj):
    shape_map = {'square': '#', 'pyramid': 'A',
                'block': 'O', 'table': 'T', None: ' '}
    shape = obj_shape(obj)
    return shape_map[shape]

def build_simple_rows(m, g):
    held_shape = draw_simple_shape(obj_in_hand())
    top_row = '<' + held_shape
    stacks = [build_stack(s) for s in squares]
    tallest = max([len(s) for s in stacks])
    lower_rows = []
    for i in range(tallest):
        row = ''
        for s in stacks:
            if i < len(s):
                row += draw_simple_shape(s[i])
            else:
                row += ' '
        lower_rows.append(row)
    display_rows = [top_row, ''] + list(reversed(lower_rows))
    return display_rows

def draw_simple_world(rows):
    print('\n'.join(rows))

If that all worked, we should just be able to look at the state of the world with `draw_simple_world(build_simple_rows(m, g))`.

In [ ]:
draw_simple_world(build_simple_rows(m, g))

## Talking to the robot ##

Let's add some interactivity, now that the world and parser are set up.
The main thing this is going to do at first is check the truth of a sentence we give it.
So, let's first define something that will check whether a sentence is true.
It takes some parses
of a sentence and checks whether the truth conditions of the first parse
are met.

In [ ]:
def check_truth(parse):
    treesem = parse.label()['SEM']
    return m.satisfy(treesem, g)

And now we can define the function that actually allows us to talk to the
robot.

Because we are going to be extending this, we will do a little trick here.
What is going to happen is that it will ask for a sentence, and then
parse it.  If it is a statement, we want it to evaluate if it is true.
But soon we will want to add something so that if it is a question, it
will evaluate its truth and answer "yes" or "no", and then something so
that if it is an imperative, it will perform the requested action.
So in order not to change the `chat()` function, we can define a
dictionary of functions to call to process the parse based on clause
type, then refer to that within `chat()`.  It's kind of a clever way
to allow it to be extended as we continue through this project.

In [ ]:
def do_declarative(parse):
    if check_truth(parse):
        print('That is true.')
    else:
        print('That is not true.')

def show_world():
    draw_simple_world(build_simple_rows(m, g))
    
type_handlers = {'decl': do_declarative}

def chat():
    print('Type bye to leave, type look to show the scene.')
    print()
    while True:
        sent = input("> ").lower()
        if sent == 'bye':
            break
        elif sent == 'look':
            show_world()
            continue
        try:
            parses = sent_parse(gramspec, sent)
            treetype = parses[0].label()['CT']
            if treetype in type_handlers:
                type_handlers[treetype](parses[0])
        except Exception as e:
            print("Error: {}".format(e))
            print("Sorry, what?")
    print("Bye!")

Take a moment to try to understand what it is doing and then give
it a spin.

In [ ]:
chat()

There are two "magic" statements you can say to the robot.  One is
'bye', which will end it (`break` exits the `while True` loop);
the other is 'look', which will draw the world and then go back and
get more input (`continue` goes back up and starts the `while True`
loop again).

If it gets an error while parsing (for example, if you use a word
it does not know), then it will print "Sorry, what?" and get more input.

## Questioning the world ##

It is a bit unnatural to just say things and have the robot confirm
whether it is or is not true.  It would be nicer if we could ask it.

All we need for yes-no questions is to fix up the parser so that it
understands when a yes-no question is asked.  The basic task the
program performs once the semantics is established is identical to
checking the truth of a statement (except it will say "Yes" instead
of "That is true").

The form of a yes-no question, in this limited grammar, is just "is DP PP?".
Specifically, the auxiliary "is" appears at the front instead of between
the DP and PP.

In [ ]:
# ynq CBAR

gramspec['ynq'] = r"""
CBAR[SEM=?tp, CT=?ct] -> VCOP TPNOAUX[SEM=?tp, CT=?ct]
TPNOAUX[SEM=<?subj(?vp)>, CT='ynq'] -> DP[SEM=?subj] VPNOAUX[SEM=?vp]
VPNOAUX[SEM=?pp] -> PP[SEM=?pp]
"""

Here, we defined a version of CBAR that has the VCOP first, and an
"TP-without-an-aux" (TPNOAUX) that has just a DP and a "VP-without-an-aux"
(VPNOAUX), which itself contains just a PP.

In [ ]:
# define the ynq handler

def do_ynq(parse):
    if check_truth(parse):
        print('Yes.')
    else:
        print('No.')

# add the handler to the list of things chat() handles
type_handlers['ynq'] = do_ynq

**ACTIVITY**. Show that it worked by asking it
"is a red block on an odd square".  How about an even square?  Play around a little bit more.

*Note:* You should not put a question mark at the end, the grammar will not know
what to do with punctuation.

> This doesn't count as part of the homework because the chat has no history. 
> But just ask the robot anyway and see if it says the right thing.

In [ ]:
chat()

## Changing the world ##

After kind of a slow start, we've made a bunch of progress pretty quickly.
We now have a world, and we can state and ask things about it, and the robot
understands to the extent that it can determine the truth of statements based
on the facts of the world.

The last major thing we'll do is add the ability to change the configuration of
the world, by adding imperatives to the parser.  This is also where we have to
add a lot more "smarts" to the system, because although NLTK was able to take
care of the heavy lifting in the domain of parsing and evaluation of logical
formulae, we need to tell the robot how to actually affect the world.

The first thing we need to do is revise the grammar to have one more clause
type, an imperative.  An imperative has a silent (implicit) subject, but
is otherwise mostly the same as a statement.  So, all we really need to do
is add one more alternative definition of TP:

In [ ]:
# imperative TP

gramspec['imp'] = r"""
TP[SEM=<?vp(hand)>, CT='imp'] -> VP[SEM=?vp]
"""

As you can
see, what it essentially does is looks for a sentence with no subject, and
assumes that "hand" (the robot's actor) is the subject, and sets the clause
type to 'imp'.

This is not quite enough to be satisfying, though, we need to add some verbs.
We are going to add just two verbs: *take* and *put*.  What we want the
robot to do if we tell it to *take* something is to put it in its hand, and
if we tell it to *put* something somewhere, it will do that.

Although we'll go ahead and fill in the semantics here, we actually are
not going to wind up using a lot of the semantics it computes.  But, here
is a little more to add.  We are adding a transitive verb "take",
which is very much like the transitive P "on", and a ditransitive
verb "put" that has both a DP and PP argument. 

In [ ]:
gramspec['acts'] = r"""
VP[SEM=<?obj(?v)>] -> V[SEM=?v] DP[SEM=?obj]
V[SEM=<\x y.take(y,x)>] -> 'take'
VP[SEM=<?v(?obj,?pp)>] -> VDT[SEM=?v] DP[SEM=?obj] PP[SEM=?pp]
VDT[SEM=<\Y X x.X(\z.Y(\y.put(x,y,z)))>] -> 'put'
"""

At this point it should already be able to parse "put a red block on an even square".  Let's try.

In [ ]:
# try parsing put a red block on an even square
print(sent_sem(gramspec, "put a red block on an even square")[0])
print(sent_parse(gramspec, "put a red block on an even square")[0].label()['CT'])

Great.
But the robot doesn't know how to make the command happen.

And this is where things get a little complicateder, because there are a lot
of things to take into account.  We are going to be building a `do_imperative`
function like the `do_declarative` and `do_ynq` functions that will take the parse
and deal with it.

The `do_imperative` function is supposed to look at the imperative
tree, determine what verb was used (*take* or *put*), and then take the
action.  We're going start with *take* because it's simpler.

Think first about what the robot is supposed to do if it is asked to
take something.  If you say "take a block" then there are several
possible choices.  The robot can just pick one.  However, you can't
take something that's underneath something else, so it needs to check for
that.  Also, taking something means putting it in its hand, but if it
is already holding something, it needs to put that thing down first.

Let's try a couple of things by hand before we try to automate anything.
First, we will parse the sentence "take a red block" by hand:

In [ ]:
parses = sent_parse(gramspec, "take a red block")
print(parses[0].label()['CT']) # should say 'imp'
print(parses[0]) # the cp
print(parses[0][0]) # the cbar
print(parses[0][0][0]) # the tp

As you can see, we're kind of working our way down the tree.  We can
get the VP like this:

In [ ]:
vp = parses[0][0][0][0]
print(vp) # the vp
print(vp[0]) # the v
print(vp[0][0]) # the word in the sentence

After a whole lot of `[0]`s, we managed to zoom into the actual verb.
Because we have a very limited grammar, this is reliable enough.  That is,
we can count on being able to figure out what verb we have by looking at
`vp[0][0][0]`.  So, in particular, we can check to see if it is "take".

Likewise, we can get the object (the thing we're asking the robot to take)
with this:

In [ ]:
obj = vp[1]
print(obj)

For the object, we actually do need to use the semantics NLTK computed
for us.  Specifically, we need to figure out what individuals in the
model the object can describe (so we know what the robot is choosing
between when it takes something).

Here, I wound up doing something a bit tricky to get this into a form
that we can use NLTK's evaluation functions for.  The logical formula for
the object is:

In [ ]:
obj_sem = obj.label()['SEM']
print(obj_sem) # \Q.exists x.(red(x) & block(x) & Q(x))

This is the right form for the subject of a sentence, it takes a
predicate (`Q`) and is true if there is an `x` such that `x` is red,
a block, and `Q` is true of it.  But our goal right now is not to
evaluate a sentence, but to find out what the objects are that this
DP can represent.  We want to know what the red blocks are.
After some reflection, the simplest thing I could come up with is
to make `Q` is kind of predicate that means "is y".
So, what we're going to be asking is: What are the values of y
that make "there is a red block that is y" true?  It's a little
bit roundabout.  (If you see a better way to do this, let me know!)

So, to implement this, we will define a predicate I'll call
`isthis`:

In [ ]:
isthis = nltk.sem.Expression.fromstring(r'\x.(x=y)')

This is true for any individual that is... this, whatever y is pointing to.
So, if we substitute `isthis` in for `Q`:

In [ ]:
this_obj = nltk.sem.ApplicationExpression(obj_sem, isthis).simplify()
print(this_obj) # exists x.(red(x) & block(x) & (x = y))

We now have an open formula (on `y`) that we can check for satisfiers on.
As I indicated, this is wandering around in the weeds a bit, but the main
thing is that if we say:

In [ ]:
options = m.satisfiers(this_obj, 'y', g)
print(options) # {'a'}

we can get a list of those individuals that are red blocks.

Having walked through that, let's just commit that to a function that will
do all of that and give us a set of options. 

In [ ]:
def find_options(obj):
    obj_sem = obj.label()['SEM']
    isthis = nltk.sem.Expression.fromstring(r'\x.(x=y)')
    this_obj = nltk.sem.ApplicationExpression(obj_sem, isthis).simplify()
    options = m.satisfiers(this_obj, 'y', g)
    return options

To make sure it worked, you can see if you get `{'a'}` for this:

In [ ]:
print(find_options(obj))

We still haven't defined the `do_imperative()` function (and so the robot
can't yet handle imperatives), but we now know better what it should look like.
It should find the VP, then the verb, and if the verb is "take", it should find
the possible objects it could be referring to.  The structure of the function
is like this:

In [ ]:
def do_imperative(parse):
    vp = parse[0][0][0]
    v = vp[0][0]
    if v == 'take':
        obj_opts = find_options(vp[1])
        if do_take(obj_opts):
            show_world()
    else:
        print("I'm unsure what you are asking me to do.")

def do_take(obj_opts):
    if len(obj_opts) == 0:
        print('There is no such object to take.')
    else:
        print('out of order.')
    return False
    
type_handlers['imp'] = do_imperative

Of course, we want to replace the "out of order" message with some actual
action (by redefining `do_take()` later).  But at this point, `chat()` should at least run, and give you an
"out of order" message if you try to "take a red block", and a
"there is no such object to take" message if you try to "take a blue table".

In [ ]:
chat()

Now, let's consider all the possible scenarios, assuming that there are
some options available for what object it could be taking.

If there are several options, it needs to pick one.  If it already has
something in its hand, it might be what it needs, in which case it
doesn't need to do anything.  But if it has something different
in its hand, it needs to put that down first, somewhere.
And it can't pick something up that's underneath something else.

Things get surprisingly complicated once we start involving the world.

Since the world is designed in such a way that it has more squares
than other objects, the robot is guaranteed to always have a place
to put down the object it is holding onto an empty square.
We can define a function
that will locate an empty square, and be confident that it'll find one.

In [ ]:
def empty_square():
    for square in squares:
        if not whats_on(square):
            return square

If you execute this function, it should find an empty square.
In fact, it should find `s3` at present.

In [ ]:
empty_square()

And we can define a function that will remove from the options any
objects that are not visible (because they are hidden under something
else).

In [ ]:
def visible_things(objects):
    return {obj for obj in objects if not whats_on(obj)}

Since the robot can't pick up the floor squares, we also want to
eliminate those from the options of pick-up-able things.  We don't
want them to be invisible/hidden (because we can put things on them),
but we can add an extra filter to remove any squares.  Since
invisible things are also not takable, we will incorporate the
visibility filter in `takable_things()` as well:

In [ ]:
def takable_things(objects):
    return {obj for obj in visible_things(objects) if 'square' not in obj_properties(obj)}

To see what we have done, run them.

In [ ]:
print(visible_things(dom))
print(takable_things(dom))

Now, to put an object down, the robot will put the object on another
object.  For right now, it will just put the object on an empty square.
This means that we remove the object from the robot's hand, and we put
it in an `on` relation to another object.  So, here is a function that makes
this happen.

In [ ]:
def put_on(target):
    obj_to_place = obj_in_hand()
    val['held'] = {}
    val['on'] |= {(obj_to_place, target)}

I used a kind of nifty little construct above in the last line there.
The `|=` operator is kind of like `+=`, it's a shorthand.
So the last line is equivalent to

```python
    val['on'] = val['on'] | {(obj_to_place, target)}
```

Since both `val['on']` and `{(obj_to_place, target)}` are sets,
the `|` operator performs a set union, meaning that it adds all of members
together and puts them in `val['on']`.  Effectively, it is adding one more
"on" relation to the model.

Lastly, to pick an object up, we put it in the robot's hand, and remove
the `on` relation that previously indicated that it was on top of something.

In [ ]:
def pick_up(obj):
    val['held'] = {(obj,)}
    val['on'] = {(x,y) for (x,y) in val['on'] if x != obj}

Those are all the pieces we need to implement *take*.

Describing the function in English is probably not much clearer than
just reading the code.  But: if there are no options, it prints a
message saying so.  (And this much we already had in the skeletal
version of `do_take()` we defined above.)  The function will default
to returning `False` (indicating the world did not change and
we do not need to redraw it), but if something happens, we will
return `True` when it does.

If there are options but one is already in hand,
print a message saying so and report success (`True`).
Otherwise, narrow the options to the takable objects (those that
are not squares and not underneath something).  If that leaves no
options, print a message saying so.  Otherwise, put the thing
currently in hand down on an empty square, pick up the first of the viable
options, and report success (`True`).  If we haven't already reported
success, then report failure (`False`).

In [ ]:
def do_take(obj_opts):
    if len(obj_opts) == 0:
        print('There is no such object to take.')
    else:
        if obj_in_hand() in obj_opts:
            print('I am already holding such a thing.')
            return True
        else:
            takable = takable_things(obj_opts)
            if len(takable) == 0:
                print('I cannot see such a thing to take.')
            else:
                put_on(empty_square())
                # take the first option
                obj_to_take = list(takable)[0]
                pick_up(obj_to_take)
                print('Object taken.')
                return True
    return False

**ACTIVITY**. Play around with it a little by running `chat()`.
It's kind of fun.  Look.  Take a block.
Look. Take a red block.  Maybe it's not all that fun.  But it's a little bit fun.

In [ ]:
chat()

The next thing to implement is *put*, but we already have a lot of those
pieces in place just from *take*.  In order to put something somewhere, the
robot must take it first and then put it in the target location.  Now the
target need not be a floor square, so we can make stacks of things.

The first step is to execute *take* on the object we are supposed to
be putting somewhere.

If the verb is *put*, then we need to determine whether we can take
the object, and then we need to determine whether the place we want to put
it is visible.  And then we do it.  The following function does this.

In [ ]:
def do_put(obj_opts, vp):
    if do_take(obj_opts):
        pp = vp[2]
        p = pp[0][0]
        loc_opts = find_options(pp[1])
        if len(loc_opts) == 0:
            print('There is no such place to put anything.')
        else:
            visible = visible_things(loc_opts) - {obj_in_hand()}
            if len(visible) == 0:
                print("I cannot see such a place to put anything.")
            else:
                # pick the first option
                target = list(visible)[0]
                put_on(target)
                print('Object placed.')
                return True
    return False

**TASK 7.** Redefine `do_imperative()` so that if will call the `do_put(obj_opts)` function if the verb is `'put'`.
You should be very much following the model for when the verb is `'take'`, it is not very complicated.  But it does
require that you're kind of following along.

In [ ]:
# Answer 7.  Redefine do_imperative(parse) to include do_put(obj_opts)



When we redefine `do_imperative` we need to add it to `type_handlers` again because its identity has changed.

In [ ]:
# because we redefined do_imperative, we need to update it in type_handlers
type_handlers['imp'] = do_imperative

Now you should be able to run `chat()` and start putting things on
other things.  Like "put a red block on a pyramid".


In [ ]:
chat()

At this point, since you have been changing the world around, you might want to
be able to just reset it to its initial state.  If you find you want to
do that, you can define and use this:

In [ ]:
def reset_world():
    val['on'] = {('a','s1'),('b','s2'),('d','s4'),('c','d')}
    val['held'] = {('e',)}

In [ ]:
reset_world()

## Improving the view and adding labels ###

Let's go back to the display.  Right now the display is very small,
and we don't know which objects are red, etc.  So, we can go back
to the world-drawing code and change it around a little bit so that the
objects are bigger shapes with labels on them.  We won't use graphics,
but we'll instead fashion "text graphics"
that represent the shapes, with a label that indicates something about the
properties the object has.  So, before we can construct the whole image of
the world, we need to be able to draw an individual block, pyramid, etc.

Conceptually, what we're going to do is replace the `draw_simple_shape()`
function that we were using before with a `draw_shape()` function that
draws larger, 8x4-character objects.

Here's the basic idea in a simple version before we do the whole thing:

In [ ]:
def draw_shape_v1(obj):
    s = r"""
/------\
|      |
|{: ^6}|
\------/
""".format('label!')
    return s.split("\n")[1:-1]

To make it easier to work with, it defines the shape as a multi-line string
using the `r"""` delimiter.  The `r` (for "raw text") is important there, because we are using
the `\` character, and we do not want it to be interpreted as "escaping" the next
character.  Notice too that we have to put the shape all the way on the left
(not indented) or else it will include a bunch of leading spaces we do not want to have.
So, we draw the shape we want, including a format string for the label in the
third line, and then insert the string "label!" in there.  The format string says that
the inserted string should be 6 characters long (minimally), and padded with spaces on either
side in order for it to be centered.  Once the multi-line string is defined in `s`, it is
then split on line breaks (`\n`), and then the first and last lines are discarded.

In [ ]:
print(draw_shape_v1('a'))

If you want to see the shape actually drawn out, join things up with lines:

In [ ]:
print("\n".join(draw_shape_v1('a')))

**TASK 8**. Why do we see so many backslashes when we did `draw_shape_v1('a')`?

*Answer 8.* (markdown)

With the proof of concept out of the way, we can now make a more sophisticated
`draw_shape(obj)` function that will check to see what type of object `obj` is, 
draw the right shape for the object, and fill in the label based on the other
(non-shape) properties.

One of those properties (block) is relevant to knowing what shape we are going
to draw, and the rest of the properties will be used for the label.  So, let's
also define a function that takes the properties and separates out the shape property,
then builds a label with the rest of the properties by using the first two letters of
at most three of them (since we have only 6 characters) to work with.  (So, a big red
block would be a block carrying a label of "bire" or "rebi".)

In [ ]:
def obj_form(obj):
    shape_properties = {'block', 'pyramid', 'table', 'square'}
    shape_property = obj_properties(obj) & shape_properties
    if len(shape_property) == 0:
        return None
    shape_name = list(shape_property)[0]
    properties = obj_properties(obj) - {'held', 'thing', shape_name}
    abbrevs = [prop[:2] for prop in properties]
    label = "".join(abbrevs)[:6]
    return(shape_name, label)

So, what's happening here?  We retrieve the object's properties, and then find
which shape property the object has (by intersecting the object properties
with a set of shape properties).  We want to disregard the "held" property
(which is true of the object that is currently in the robot hand), because
we don't want that as part of the label.  Same for "thing", which is a
property of all of the shapes, and for the shape property itself.
We then take the rest of the properties and
extract the first two letters of each property name into a list, join them
together, cut it off at 6 characters (in case it would have been longer).
We then return a pair, the first member of which is the shape property, and
the second member of which is the label.  If there is no shape property, it
returns `None`.

**TASK 9**. Try this out.  What is the shape and label of object `a`?

In [ ]:
# Answer 9: What is the shape and label of object 'a'?


Now, we're all set to finish up the `draw_shape()` function so that it
can draw more than just a block.  There are two special shapes that we
want to be able to draw apart from blocks, pyramids, squares, and tables:
the robot hand, and an empty space.  So, we're going to set up
`draw_shape()` to respond to a couple of "magic" values.  If we send it
`"HAND"` as the object, it will draw the hand, and if we send it `None`
it will draw an empty space.  Otherwise, it will look up the object
in the model and get its properties and label.

A note about the code below: *There are spaces* after the right edge, and under the hand,
since every line in the multi-line string needs to be 8 characters long.

In [ ]:
def draw_shape(obj):
    if obj:
        if obj == "HAND":
            s = r"""
       /
======<=
       \
        
"""
        else:
            (shape, label) = obj_form(obj)
            if shape == 'block':
                s = r"""
/------\
|      |
|{: ^6}|
\------/
""".format(label)
            elif shape == 'pyramid':
                s = r"""
   /\   
  /  \  
 /    \ 
/{:_^6}\
""".format(label)
            elif shape == 'table':
                s = r"""
|------|
|      |
|{: ^6}|
|      |
""".format(label)
            else:
                s = r"""
========
|======|
|{:=^6}|
|======|
""".format(label)
        return s.split("\n")[1:-1]
    else:
        return [' '*8]*4

To make sure it worked, try `draw_shape('a')` and `draw_shape('HAND')`.

In [ ]:
draw_shape('a')

In [ ]:
draw_shape('HAND')

Now, we want to integrate this into our drawing procedure.  Most of the
previous `build_simple_rows()` remains logically unchanged, except that when
it comes time to translate the rows into display strings, we have 4x8 lists
of strings instead of a simple character.

So, we replace `row_string()` with `row_tiles()`:

In [ ]:
def row_tiles(row_objects):
    return [draw_shape(x) for x in row_objects]

And then `build_rows()` that makes use of `row_tiles()`.

In [ ]:
def build_rows(m, g):
    stacks = [build_stack(s) for s in squares]
    tallest = max([len(s) for s in stacks])
    rows = []
    for i in range(tallest):
        row = []
        for s in stacks:
            if i < len(s):
                row.append(s[i])
            else:
                row.append(None)
        rows.append(row)
    rows += [[None], ['HAND', obj_in_hand()]]
    display_rows = [row_tiles(row) for row in reversed(rows)]
    return display_rows

And then instead of `draw_simple_world()`, which just printed each row
string, we want to print each row group (since each representation is now
4 rows tall).

In [ ]:
def draw_world(rows):
    for row in rows:
        for line in range(4):
            print(''.join([col[line] for col in row]))

And now to draw the enhanced world, you can just do this:

In [ ]:
draw_world(build_rows(m,g))

So, how do we get the robot to use our better drawing procedure?

It turns out, we thought ahead.  The robot uses `show_world()` to
draw the world, which itself called `draw_simple_world()`.
But now, if we redefine `show_world()` to use `draw_world()` instead,
we'll be using the new better world drawing function automatically.

In [ ]:
# we can redefine what show_world() displays, which chat() calls
def show_world():
    draw_world(build_rows(m, g))

Once you've done that, you can run `chat()` and play around fairly effectively
in this block world.  You can now take a red block, put a red block on
a table, ask if a red block is on an odd square, or check the truth of
a red block is on an odd square. 

In [ ]:
chat()

## Further directions

This is already very long, so we won't do much else here.
But there a bunch of things that would be neat to add, and are kind of
within reach.

 - the real SHRDLU had a planning module, so that if you
said you wanted to pick up something that was covered, it would move
things out of the way so it could get it.  That would be relatively easy to
implement.

 - We could also add *wh*-questions, to handle things like
"what is on the blue pyramid" or "how many blocks are on even squares" or
"where is the blue pyramid".

 - Our support for the universal quantifier is
a bit incomplete as well; you can ask "is every block on an even square"
but you can't "put every pyramid on a block".  This could be shored up,
though it's actually a bit of a harder problem.

 - SHRDLU also had the ability
to learn things like that a stack of a block and pyramid can be called a
"steeple" (and then "steeples" can be referred to from then on), and could
absorb facts like "I like red blocks" and then answer questions about them.
Some kind of memory of names like this could be added.

 - Another easy-ish fix we could add is to deal with the fact that our robot
only knows singulars, not plurals.

 - We could add handling for
definite noun phrases.  At the moment, our robot treats "the pyramid" and
"a pyramid" exactly the same (except that it marks "the pyramid" as definite).
But if we ask the robot to do something with "the block" it should be unsure
which block we meant.  Unless we had just been interacting with a block
recently, in which case we can assume that it was that block we meant.

## Pyramids are pointy ##

Here is just one addition for you to do, without much explicit guidance.
The task is to prevent the robot from putting anything on a pyramid.

> You can just make this say "I can't put that there" or something if
an attempt is made.  That would be fine.  You could also try to imagine
different scenarios, such that if you try to put a block on a pyramid
it falls into an adjacent column.  Unless that column already has so much
stuff in it that it can't fall.  Can you put something on a pyramid if it is
between two taller columns?  What if you later move one of the supports?
It's very easy to go overboard.  But you can make it just refuse to put
something on a pyramid, that's really all we need.

**TASK 10**.  Modify `do_put` so that if prevents you from putting anything
on a pyramid.

In [ ]:
# Answer 10: Modify do_put so that it prevents you from putting anything on a pyramid



In [ ]:
chat()